In [ ]:
def next_wstation(workstation: str) -> str:
    """
    Get the next workstation based on the current one.
    :param workstation: Current workstation
    :return: Next workstation
    """
    id = workstation[-1]
    next_id = int(id) + 1
    if next_id > 3:
        return None  # No more workstations
    
    return f"W{next_id}"

# Read the file and parse the events
filename = "group03.txt"
filter = "W2"  # Set to "W1","W2" or "W3" to filter the data per workstation, False to not filter

# Load data
events = []
if filename.startswith("group03"):
    with open(filename, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            time = int(parts[0])
            lot = parts[1]
            event_type = parts[2]
            if event_type == 'Created':
                workstation = 'W0'  # initial workstation
                event_type = 'D'
            elif event_type[0] == 'W':
                workstation = parts[2]
                event_type = parts[3]
            # Extract lot number from lot name
            lot_number = int(lot.replace('lot', ''))
            if workstation == filter:
                events.append((time, workstation, event_type, lot_number))
            # add corresponding arrival event at next workstation
            next_workstation = next_wstation(workstation)
            if next_workstation !=  None and next_workstation == filter:
                events.append((time, next_workstation, 'A', lot_number))

else:
    with open(filename, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            time = int(parts[0])
            lot = parts[1]
            event_type = parts[2]
            # Extract lot number from lot name
            lot_number = int(lot.replace('lot', ''))
            workstation = 'W1'
            events.append((time, workstation, event_type, lot_number))


# EPT Algorithm

def detOvert(xs, i, arrived_before_start):
    """
    Determine the overtaking status of lots in the system.
    :param xs: List of tuples (arrival_number, aw) for lots in the system
    :param i: Arrival number of the departing lot
    :return: Tuple containing the updated list of lots, number of overtaken lots, and WIP level at arrival
    """
    ys = []  # List to store lots that remain after processing
    overtaken_count = 0  # Number of overtaken lots
    aw = 0  # WIP level at the time of the departing lot's arrival
    idx = 0

    for idx, (j, wip) in enumerate(xs):
        if j < i:
            ys.append((j, wip))  # Keep lots that arrived before the departing lot
        elif j == i:
            aw = wip  # WIP level when the departing lot arrived
            # Count all lots that arrived after this lot but departed earlier
            overtaken_count = len(xs[:idx])  # Lots before this index are overtaken
            break

    # Remove the departing lot from the list
    ys.extend(xs[idx + 1:])

    return ys, overtaken_count + len(arrived_before_start), aw

xs = []  # list of (arrival_number, aw)
s = None  # EPT start time
sw = None  # WIP-level after EPT start

# Outputs
EPT_results = []  # (EPT_duration, WIP-level)
Overtaking_results = []  # (number_overtaken, aw)

arrived_lots = []  # List to keep track of lots that arrived before EPT start
departed_lots = []  # List to keep track of departed lots

prev_w2_depture = None  # Previous departure time for W2
prev_lot = None  # Previous lot number for W2
w2_arrival_times = []  # List to keep track of W2 arrival times
batches = []  # List to keep track of batches for W2
batch_arrived = []  # List to keep track of lots that arrived in the current batch
current_batch = []  # Current batch for W2

for (tau, wstation, ev, i ) in events:
    if ev == 'A':
        arrived_lots.append(i)  # Keep track of lots that arrived before EPT start
    elif ev == 'D':
        departed_lots.append(i)  # Keep track of departed lots

    # check batches for workstation W2
    if wstation == 'W2' and ev == 'D':
        if prev_w2_depture == tau or prev_w2_depture is None:
            current_batch.append(i)  # Add to current batch
        else:
            batches.append(current_batch)
            for lot in w2_arrival_times.copy():
                if lot[0] == prev_lot:
                    batch_arrived.append(lot[1])  # Add the batch arrival time
                    w2_arrival_times.remove(lot)  # Remove the lot from the list
                if lot[0] in current_batch[0:-1]:
                    w2_arrival_times.remove(lot)  # Remove the lot from the list

            current_batch = [i]  # Start a new batch
        prev_w2_depture = tau
        prev_lot = i  # Update the previous lot

    if wstation == 'W2' and ev == 'A':
        w2_arrival_times.append([i, tau])  # Keep track of W2 arrival times

arrived_before_start = set(departed_lots) - set(arrived_lots)  # Lots that arrived before EPT start and have not departed
if current_batch:
    batches.append(current_batch)  # Add the last batch if it exists


for (tau, wstation, ev, i) in events:
    if ev == 'A':
        arrived_lots.append(i)  # Keep track of arrived lots
        if len(xs) == 0:
            s, sw = tau, 1
        xs.append((i, len(xs)))
    elif ev == 'D':
        # End current EPT
        if i in arrived_before_start:
            arrived_before_start.remove(i)
            continue

        if wstation == 'W2':
            for (batch, arrival) in zip(batches, batch_arrived):
                if i in batch:
                    s = arrival  # Set EPT start time to the batch arrival time
                    if i == batch[-1]:
                        batches.remove(batch)
                        batch_arrived.remove(arrival)
                    break
        
        EPT_duration = tau - s
        EPT_results.append((EPT_duration, sw))
        
        xs, k, aw = detOvert(xs, i, arrived_before_start)
        Overtaking_results.append((k, aw, i))

        if len(xs) > 0:
            s, sw = tau, len(xs)

# Display Results
print("EPT Durations and WIP-levels:")
for duration, wip in EPT_results:
    print(f"EPT Duration: {duration}, WIP-level: {wip}")

print("\nOvertaking Details:")
for overtaken, aw, i in Overtaking_results:
    print(f"Lot: {i} Number of overtaken lots: {overtaken}, WIP before arrival: {aw}")

# Calculate and display mean EPT duration
mean_EPT = sum(duration for duration, _ in EPT_results) / len(EPT_results)
print(f"\nMean EPT Duration: {mean_EPT:.2f}")


ValueError: list.remove(x): x not in list